# Flat Likelihood, Flat Prior
*by* **David W. Hogg** (NYU) (MPIA) (Flatiron)

## Goals:
Demonstrate that you can have a peak in your posterior even when your likelihood has no peak, and your priors are flat.

## To-do:
- Make publication-worthy plots.
- Write up correctly in the document.

In [ ]:
import numpy as np
from scipy.special import logsumexp
import pylab as plt
import emcee
import corner
RNG = np.random.default_rng(17)
plt.rcParams.update({
    "text.usetex": True,
    "figure.figsize": (2, 2),
})

In [ ]:
N, P, Q = 12, 8, 2

In [ ]:
def design_matrix(n, p, q, rng=RNG):
    assert q <= p < n
    return (rng.integers(-4, 4, size=(n, q)) @ rng.integers(-4, 4, size=(q, p))).astype(float)

def log_like(a, X, ys, ivars):
    resids = ys - X @ a
    return -0.5 * resids @ (ivars * resids)

def wls(X, ys, ivars):
    return np.linalg.lstsq(X.T @ (ivars[:, None] * X), X.T @ (ivars * ys), rcond=None)[0]

def log_profile_like(ai, i, X, ys, ivars):
    Xi = np.delete(X, (i), axis=1)
    a_hat = wls(Xi, ys - X[:, i] * ai, ivars)
    a = np.insert(a_hat, i, ai)
    return log_like(a, X, ys, ivars)

PRIOR_LIMITS = np.zeros((P, 2))
PRIOR_LIMITS[:, 0] = -5.
PRIOR_LIMITS[:, 1] = 5.
PRIOR_LIMITS[0] = -16., 16.
def log_prior(a):
    if np.any(a < PRIOR_LIMITS[:, 0]):
        return -np.Inf
    if np.any(a > PRIOR_LIMITS[:, 1]):
        return -np.Inf
    return 0.

def log_post(a, X, ys, ivars):
    lnpi = log_prior(a)
    if np.isfinite(lnpi):
        return lnpi + log_like(a, X, ys, ivars)
    else:
        return -np.Inf

def mh_mcmc_step(log_post, a, proposal, X, ys, ivars, rng=RNG):
    a_new = a + proposal * rng.normal(size=a.shape)
    lr = np.log(rng.uniform())
    if log_post(a_new, X, ys, ivars) - log_post(a, X, ys, ivars) > lr:
        return a_new
    else:
        return a

In [ ]:
# make and check design matrix
X = design_matrix(N, P, Q)
print(X.shape)
u, s, v = np.linalg.svd(X)
print(s)

In [ ]:
# print design matrix in latex format
print(r"X = \begin{bmatrix}")
for x in X:
    print(" & ".join((r"{:3.0f}.".format(xx) for xx in x)) + r" \\")
print(r"\end{bmatrix}")

In [ ]:
a_true = RNG.normal(size=P)
ys_true = X @ a_true
ivars = 100. * np.ones_like(ys_true)
ys = ys_true + RNG.normal(size=N) / np.sqrt(ivars)

In [ ]:
a_hat = wls(X, ys, ivars)
np.linalg.cond(X.T @ (ivars[:, None] * X))

In [ ]:
plt.errorbar(np.arange(N), ys, yerr=1/np.sqrt(ivars), color="k", fmt="o")
plt.errorbar(np.arange(N), X @ a_hat, color="r", fmt="o", mfc="none")

In [ ]:
# make and plot profile likelihoods
ays = np.arange(PRIOR_LIMITS[0,0], PRIOR_LIMITS[0,1]+0.001, 1.0)
lpls = np.zeros_like(ays)
for k, ai in enumerate(ays):
    lpls[k] = log_profile_like(ai, 0, X, ys, ivars)
profile_like_ratios = np.exp(lpls - np.max(lpls))
plt.plot(ays, profile_like_ratios, color="k")
plt.xlim(PRIOR_LIMITS[0])
plt.xlabel(r"$\theta$")
plt.ylim(-0.1, 1.1)
plt.ylabel("ratio to maximum")
plt.axhline(0., color="k", lw=0.5, alpha=0.5)
plt.title(r"profile likelihood for $\theta$")
plt.savefig("profile_likelihood.png", dpi=200)

In [ ]:
# make posterior sampling
nwalkers, nsample = 100, 10000
p0 = 0.01 * RNG.normal(size=(nwalkers, P))
sampler = emcee.EnsembleSampler(nwalkers, P, log_post, args=[X, ys, ivars])
state = sampler.run_mcmc(p0, 1000) # burn in
sampler.reset()
state = sampler.run_mcmc(state, nsample)

In [ ]:
# plot all-parameters posterior sampling
a_samples = sampler.get_chain(flat=True)
f = plt.figure(figsize=(10, 10))
ff = corner.corner(a_samples,
                   range=PRIOR_LIMITS,
                   labels=(r"$\theta$", ) + tuple((r"$\alpha_{}$".format(p) for p in range(1,P))),
                   fig=f)
plt.savefig("posterior_full.png", dpi=200)

In [ ]:
# plot parameter-of-interest posterior
post, _ = np.histogram(a_samples[:,0], bins=ays, density=True)
print(post)
plt.step(0.5 * (ays[1:] + ays[:-1]), post, where="mid", color="k")
plt.xlabel(r"$\theta$")
plt.xlim(PRIOR_LIMITS[0])
plt.ylabel("numerically estimated pdf")
plt.ylim(-0.1 * np.max(post), 1.1 * np.max(post))
plt.axhline(0., color="k", lw=0.5, alpha=0.5)
plt.title(r"posterior pdf for $\theta$")
plt.savefig("posterior.png", dpi=200)